In [2]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import importlib

from scipy.io.wavfile import write
import IPython

sys.path.append('../')
sys.path.append('../datasets')
sys.path.append(os.path.join(os.getcwd(), '../models/'))

import ai8x
from kws20 import KWS
import kws20

In [3]:
data_path = '/home/emretopcu/ai8x-training/data'

class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

simulate = False
args = Args(act_mode_8bit=simulate)

_ ,test_set = kws20.KWS_20_get_datasets((data_path, args), load_train=False, load_test=True)

Exception ignored in: <_io.FileIO name=65 mode='rb' closefd=True>
Traceback (most recent call last):
  File "/home/emretopcu/ai8x-training/venv/lib/python3.8/site-packages/audioread/__init__.py", line 99, in available_backends
    if ffdec.available():
Exception ignored in: <_io.FileIO name=67 mode='rb' closefd=True>
Traceback (most recent call last):
  File "/home/emretopcu/ai8x-training/venv/lib/python3.8/site-packages/audioread/__init__.py", line 99, in available_backends
    if ffdec.available():


No key `stretch` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]
Processing the label: backward. 1 of 36
Finished in 0.358 seconds.
Processing the label: bed. 2 of 36
Finished in 0.324 seconds.
Processing the label: bird. 3 of 36
Finished in 0.333 seconds.
Processing the label: cat. 4 of 36
Finished in 0.325 seconds.
Processing the label: dog. 5 of 36
Finished in 0.339 seconds.
Processing the label: down. 6 of 36
Finished in 0.627 seconds.
Processing the label: eight. 7 of 36
Finished in 0.609 seconds.
Processing the label: five. 8 of 36
Finished in 0.656 seconds.
Processing the label: follow. 9 of 36
Finished in 0.253 seconds.
Processing the label: forward. 10 of 36
Finished in 0.250 seconds.
Processing the label: four. 11 of 36
Finished in 0.603 seconds.
Processing the label: go. 12 of 36
Finished in 0.624 seconds.
Processing the label: happy. 13 of 36
Finished in 0.331 seconds.
Processing the label: house. 14 of 36
Finished in 0.343 seconds.
Processing the lab

In [ ]:
#print(f'{len(train_set)}')
print(f'{len(test_set)}')

In [ ]:
#input_length = 16384
#transform = torchaudio.transforms.TimeStretch(rate)
#audio2 = transform(audio)
#if len(audio2) > input_length:
#    audio2 = audio2[:input_length]
#else:
#    audio2 = np.pad(audio2, (0, max(0, input_length - len(audio2))), "constant")

In [ ]:
classes = ['up', 'down', 'left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one',
                   'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero',
                   'UNKNOWN']

In [ ]:
def plot_audio(data, target):
    plt.plot(data)
    plt.title(classes[target])
    plt.ylabel("Amplitude")
    plt.xlabel("Time")
    plt.show()

In [ ]:
def random_sample(data_type):
    if data_type == 'train':
        sample_index = np.random.randint(0,len(train_set))

        sample, target = train_set[sample_index]
        print(f'target: {target}, sample index: {sample_index}')
    if data_type == 'test':
        sample_index = np.random.randint(0,len(test_set))

        sample, target = test_set[sample_index]
        print(f'target: {target}, sample index: {sample_index}')
    return sample, target, sample_index
    

In [ ]:
def snr_mixer2(audio, snr):
    """
    Mix clean and noise with provided SNR level.
    """
    eps = 1e-8
    audio = torch.from_numpy(audio)
    signal_pwr = torch.var(audio)
    snr_linear = torch.tensor(10**(snr / 10), dtype=torch.float32)
    n_var = signal_pwr / snr_linear
    noise = torch.randn(audio.shape) * torch.sqrt(n_var)
    noisy_signal = audio + noise
    max_abs_noise = torch.max(torch.abs(noisy_signal))

    # Normalize the noise tensor
    if max_abs_noise > 1:
        noisy_signal /= max_abs_noise
    
    return noisy_signal

In [ ]:
snr_low = torch.tensor(20, dtype=torch.int32)
snr_high = torch.tensor(51, dtype=torch.int32)

In [ ]:
strategy_a = os.path.join("..", ".." ,"ai8x-synthesis", "trained", "ai85-kws20-qat8.pth.tar")
strategy_b = os.path.join("..", "logs", "2023.08.22-164553", "qat_best.pth.tar")
strategy_c = os.path.join("..", "logs", "yeni_yeni_merve", "qat_best.pth.tar")
mod = importlib.import_module("ai85net-kws20")

In [ ]:
strategy_a

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working with device:", device)

ai8x.set_device(device=85, simulate=False, round_avg=False)
qat_policy = {'start_epoch': 10, 'weight_bits': 8, 'bias_bits': 8}

In [ ]:
model = mod.AI85KWS20Net(num_classes=len(classes), num_channels=128, dimensions=(128, 1), bias=False, 
                           quantize_activation=False)

In [ ]:
checkpoint = torch.load(strategy_a)
model.load_state_dict(checkpoint['state_dict'], strict=False)
ai8x.update_model(model)


In [ ]:
checkpoint

In [ ]:
def evaluate(model, test_data, true_labels):
    model.to(device)
    model.eval()

    test_data = torch.from_numpy(test_data).float().to(device) 
    true_labels = true_labels.to(device)

    with torch.no_grad():
        predictions = model(test_data)
    
    predicted_classes = torch.argmax(predictions, dim=1)
    print(predicted_classes[2500:2800])

    accuracy = (predicted_classes == true_labels).float().mean()
    print(f"Accuracy: {accuracy.item()}")

In [ ]:
def _evaluate(model, test_data):
    model.eval()

    #test_data = torch.from_numpy(test_data).float()

    with torch.no_grad():
        for validation_step, (inputs, targets) in enumerate(test_set):
    with torch.no_grad():
        predictions = model(test_data)
    print(test_data[0])
    print(test_data[0].shape)
    
    
    predicted_classes = torch.argmax(predictions, dim=1)
    #print(predicted_classes[2300:2800])

    accuracy = (predicted_classes == true_labels).float().mean()
    print(f"Accuracy: {accuracy.item()}")

In [ ]:
import torchnet.meter as tnt

In [ ]:
device

In [ ]:
model.to(device)
classerr = tnt.ClassErrorMeter(accuracy=True, topk=(1, min(len(classes), 5)))
model.eval()

with torch.no_grad():
    for validation_step, (inputs, targets) in enumerate(test_set):
        #inputs = snr_mixer2(inputs,50)
        inputs = inputs.unsqueeze(0)
        inputs = inputs.to(device)
    
        
        # compute output from model
        output = model(inputs)
        classerr.add(output, targets)
        if validation_step % 1000 == 0:
            print("Batch",validation_step)
            print("Accuracy:", classerr.value()[0])

In [ ]:
print("Accuracy:", classerr.value()[0])

In [ ]:
print("Accuracy:", classerr.value()[0])

In [ ]:
print(labels[4000:4500])

In [ ]:
model.to(device)
classerr = tnt.ClassErrorMeter(accuracy=True, topk=(1, min(len(classes), 5)))
model.eval()

with torch.no_grad():
    for validation_step, (inputs, targets) in enumerate(test_set):
        inputs = inputs.unsqueeze(0)
        inputs = inputs.to(device)
    
        
        # compute output from model
        output = model(inputs)
        classerr.add(output, targets)
        if validation_step % 1000 == 0:
            print("Batch",validation_step)
            print("Accuracy:", classerr.value()[0])


In [ ]:
print("Accuracy:", classerr.value()[0])

In [ ]:
print("Whole dataset contains---",len(test_set),"---samples.")
print("Each sample is a tuple of---",len(test_set[0]),"---elements.")
print("Tuple's first element contains height of each element is---",len(test_set[0][0]),"---long.")
print("Width of each element is---",len(test_set[0][0][0]),"---long.")
print("Tuple's second element contains labels---",test_set[0][1])

In [ ]:
test_set[0]

In [ ]:
for db in range(snr_low, snr_high, 1):

    len_noisy_test_set = (len(test_set), 128, 128)
    noisy_test_set = np.zeros(len_noisy_test_set)

    for i, signal in enumerate(test_set):
        noisy = snr_mixer2(signal[0], db)
        noisy_test_set[i] = noisy
        
        if i % 1000 == 0:
            print(i)
    break
        
    evaluate(model, noisy_test_set,test_set[0][1])
    print("-------------------------------------------------------------------")
    
    


In [ ]:
# Use a list comprehension to extract the labels
labels = [label for _, label in test_set]


In [ ]:
test_set_samples = [sample for sample, _ in test_set]

In [ ]:
stacked_tensor = torch.stack(test_set_samples)
test_set_samples = stacked_tensor.numpy()

In [ ]:
noisy_test_set

In [ ]:
test_set_samples

In [ ]:
labels = torch.tensor(labels, dtype=torch.int64)

In [ ]:
print(labels[2500:2800])

In [ ]:
evaluate(model, test_set_samples, labels)

In [ ]:
evaluate(model, noisy_test_set, labels)

In [ ]:
print(len(test_set[0][0][127]))
print(test_set[0][0].shape)


In [ ]:
sample, target, sample_index = random_sample('test')

audio_sample = np.array(sample)
audio_sample = audio_sample.flatten('F')
plot_audio(audio_sample, target)
print(len(audio_sample))
print(audio_sample.shape)
print(type(audio_sample))
print(max(audio_sample))
print(np.mean(audio_sample))

#7925

In [ ]:
sample, target, sample_index = random_sample('test')
audio_sample = np.array(test_set[5000][0])

audio_sample_noisy = snr_mixer2(audio_sample, 3)

audio_sample_noisy = np.array(audio_sample_noisy)

print(audio_sample_noisy.shape)
print(audio_sample.shape)
audio_sample = audio_sample.flatten('F')
audio_sample_noisy = audio_sample_noisy.flatten('F')
plot_audio(audio_sample, test_set[sample_index][1])
plot_audio(audio_sample_noisy, test_set[sample_index][1])
#audio_sample_noisy
#print(len(audio_sample))
print(audio_sample.shape)
#print(type(audio_sample))
##print(max(audio_sample))
#print(np.mean(audio_sample))

In [ ]:
write('/home/emretopcu/ai8x-training/notebooks/test.wav', 16384, audio_sample)
IPython.display.Audio('/home/emretopcu/ai8x-training/notebooks/test.wav', rate = 16384)

In [ ]:
write('/home/emretopcu/ai8x-training/notebooks/test.wav', 16384, audio_sample_noisy)
IPython.display.Audio('/home/emretopcu/ai8x-training/notebooks/test.wav', rate = 16384)